In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import matplotlib.pyplot as plt

import math
import gzip
import pickle as pkl
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import random
random.seed(123)
import os
import time
import tflearn

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from multiprocessing import Process

from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization, LeakyReLU
from keras import optimizers
from keras import losses

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer 

import tensorflow as tf

import itertools
import gzip
import pickle as pkl

import fonctions
import batch_processing
import metric_loading

from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join

from scipy.sparse import hstack
from keras import backend as K
from sklearn.metrics import log_loss
from tensorflow import ones_like, equal, log
from keras import backend as K
from tensorflow.python import mul
import tensorflow as tf

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [4]:
def read_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id
    
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    y=df.filter( labels )

    X = df.drop(labels + [ 'mean_daily_author_presence','std_daily_author_presence',
        'language','tweet_id','tweet_timestamp','tweet_utctimestamp','engaged_with_user_account_creation', 'text_tokens',
                          'present_links','present_domains','hashtags','engaging_user_account_creation', 'present_media', 'engaged_with_user_id', 'engaging_user_id' ],axis=1)    
    
    X = X[ ['engaged_with_user_follower_count','engaged_with_user_following_count',
    'engaged_with_user_is_verified','engaging_user_follower_count','engaging_user_following_count',
    'engaging_user_is_verified','engagee_follows_engager',
    'hour_ 00','hour_ 01','hour_ 02','hour_ 03','hour_ 04',
    'hour_ 05','hour_ 06','hour_ 07','hour_ 08','hour_ 09',
    'hour_ 10','hour_ 11','hour_ 12','hour_ 13','hour_ 14',
    'hour_ 15','hour_ 16','hour_ 17','hour_ 18','hour_ 19','hour_ 20',
    'hour_ 21','hour_ 22','hour_ 23','day_0','day_1','day_2','day_3',
    'day_4','day_5',
    'day_6','user_account_age','user_presence','author_account_age',
    'author_presence','tweet_type_Quote','tweet_type_Retweet','tweet_type_TopLevel',
    'photo','video','gif','nb_hashtags','nb_links','nb_domains','len_tweet'] ] 
    
    return X, y

def save_model(model_id, classifier):
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/model_multilabel_NN_{}.pkl.gz'.format(model_id),'wb') as f:
        pkl.dump(classifier,f)
        
    return True

def build_multilabel_NN(nb):
    
    classifier = Sequential()
    classifier.add(Dense( nb+20, input_dim=nb ,activation='relu' ))
    classifier.add(Dropout(0.3))
    classifier.add(Dense( nb,activation='relu' ))
    classifier.add(Dense( nb-15,activation='relu' ))
    classifier.add(Dense( nb-30,activation='relu' ))
    classifier.add(Dense( 15,activation='relu' ))
    classifier.add(Dense(4, activation='sigmoid'))
    opt = optimizers.Adam()
    classifier.compile(optimizer = opt, loss=['binary_crossentropy'] ) 
    
    return classifier

def custom_loss(y_true,y_pred):
    
    substr = y_pred[:,1] - y_pred[:,2]
    min_value = K.constant(-1, substr.dtype.base_dtype)
    zeros = K.constant(0, substr.dtype.base_dtype)
    x = K.clip(substr, min_value, zeros)
   
    bce = tf.keras.losses.BinaryCrossentropy()
    
    total_loss =  bce(y_true, y_pred) +  100000 * K.sum(  K.square(x) ) 
    
    return total_loss


def train_on_chunk(chunk, model_id, classifier, nb_epochs):

    directory = '/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
    sc = StandardScaler()

    for _ in range(nb_epochs):

        iteration=1
        for batch_file in chunk:

            print(iteration)
            X, y = read_batch(directory, batch_file )
            X_train, y_train = shuffle(X, y, random_state=0)

            X_train = sc.fit_transform(X_train) 
            classifier.fit(X_train, y_train, verbose=1)
        
            iteration=iteration+1
            
    save_model(model_id, classifier)

    return classifier

In [5]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
chunk = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]

model_id=1005

classifier = build_multilabel_NN(52)

clf = train_on_chunk(chunk, model_id, classifier, 1)

1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
30000/30000 [==============================] - 1s 45us/step - loss: 0.2923
2
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2762
3
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2755
4
Epoch 1/1
30000/30000 [==============================] - 1s 41us/step - loss: 0.2732
5
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2690
6
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2681
7
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2677
8
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2676
9
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2702
10
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2723
11
Epoch 1/1
30000/30000 [==============================] - 1s 40us/step - loss: 

30000/30000 [==============================] - 1s 38us/step - loss: 0.2617
92
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2637
93
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2630
94
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2651
95
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2642
96
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2636
97
Epoch 1/1
30000/30000 [==============================] - 1s 39us/step - loss: 0.2652
98
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2621
99
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2652
100
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2636
101
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2634
102
Epoch 1/1
30000/30000 [================

30000/30000 [==============================] - 1s 37us/step - loss: 0.2660
184
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2612
185
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2643
186
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2601
187
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2610
188
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2622
189
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2605
190
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2661
191
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2630
192
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2631
193
Epoch 1/1
30000/30000 [==============================] - 1s 38us/step - loss: 0.2635
194
Epoch 1/1
30000/30000 [========

30000/30000 [==============================] - 1s 36us/step - loss: 0.2648
276
Epoch 1/1
30000/30000 [==============================] - 1s 36us/step - loss: 0.2608
277
Epoch 1/1
30000/30000 [==============================] - 1s 45us/step - loss: 0.2622
278
Epoch 1/1
30000/30000 [==============================] - 1s 44us/step - loss: 0.2639
279
Epoch 1/1
30000/30000 [==============================] - 1s 42us/step - loss: 0.2620
280
Epoch 1/1
30000/30000 [==============================] - 1s 48us/step - loss: 0.2605
281
Epoch 1/1
30000/30000 [==============================] - 1s 46us/step - loss: 0.2627
282
Epoch 1/1
30000/30000 [==============================] - 1s 44us/step - loss: 0.2614
283
Epoch 1/1
30000/30000 [==============================] - 1s 34us/step - loss: 0.2641
284
Epoch 1/1
30000/30000 [==============================] - 1s 32us/step - loss: 0.2616
285
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2624
286
Epoch 1/1
30000/30000 [========

30000/30000 [==============================] - 1s 36us/step - loss: 0.2645
368
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2622
369
Epoch 1/1
30000/30000 [==============================] - 1s 34us/step - loss: 0.2617
370
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2622
371
Epoch 1/1
30000/30000 [==============================] - 1s 33us/step - loss: 0.2617
372
Epoch 1/1
30000/30000 [==============================] - 1s 33us/step - loss: 0.2625
373
Epoch 1/1
30000/30000 [==============================] - 1s 36us/step - loss: 0.2600
374
Epoch 1/1
30000/30000 [==============================] - 1s 36us/step - loss: 0.2595
375
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2602
376
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2615
377
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2592
378
Epoch 1/1
30000/30000 [========

30000/30000 [==============================] - 1s 36us/step - loss: 0.2611
460
Epoch 1/1
30000/30000 [==============================] - 1s 41us/step - loss: 0.2634
461
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2603
462
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2617
463
Epoch 1/1
30000/30000 [==============================] - 1s 34us/step - loss: 0.2630
464
Epoch 1/1
30000/30000 [==============================] - 1s 36us/step - loss: 0.2620
465
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2627
466
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2579
467
Epoch 1/1
30000/30000 [==============================] - 1s 37us/step - loss: 0.2625
468
Epoch 1/1
30000/30000 [==============================] - 1s 35us/step - loss: 0.2644
469
Epoch 1/1
30000/30000 [==============================] - 1s 40us/step - loss: 0.2629
470
Epoch 1/1
30000/30000 [========

In [8]:
def load_model(model_id):
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/model_multilabel_NN_{}.pkl.gz'.format(model_id),'rb') as f:
        clf = pkl.load(f)
    return clf

def validation_on_chunk(clf, model_id, chunk, chunk_id):

    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    sc = StandardScaler()
    directory = '/home/maxime/Desktop/RecSys2020/data/val_pr_batches2/'
    iteration=1
    
    for test_batch in chunk:
        print(iteration)
        
        X_test, y = read_batch(directory, test_batch)        
        d = {'Tweet_Id': [ x.split('-')[0] for x in X_test.index ], 'User_Id': [ x.split('-')[1] for x in X_test.index ],
            'truth_like_timestamp':y.like_timestamp,'truth_retweet_timestamp':y.retweet_timestamp,
              'truth_retweet_with_comment_timestamp':y.retweet_with_comment_timestamp,'truth_reply_timestamp':y.reply_timestamp
            }
        result = pd.DataFrame(data=d)
        X_test = sc.fit_transform(X_test) 
        y_pred = clf.predict_proba(X_test, verbose=1)
        y_pred = y_pred.astype('float64')

        for idx, engagement in enumerate(engagements):
            result['Prediction'] = y_pred[:,idx]
            result.to_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/{}_{}.csv'.format(engagement, model_id),
            header=None, index=None, sep=',', mode='a')
        
        iteration=iteration+1
  
    return True



In [ ]:
eval_batch_path='/home/maxime/Desktop/RecSys2020/data/val_pr_batches2'
eval_batch_list = [f for f in listdir(eval_batch_path) if isfile(join(eval_batch_path, f))]
model_id = 1005

validation_on_chunk(clf, model_id, eval_batch_list, 0)

1
30000/30000 [==============================] - 0s 12us/step
2
30000/30000 [==============================] - 0s 12us/step
3
30000/30000 [==============================] - 0s 11us/step
4
30000/30000 [==============================] - 0s 11us/step
5
30000/30000 [==============================] - 0s 11us/step
6
30000/30000 [==============================] - 0s 11us/step
7
30000/30000 [==============================] - 0s 11us/step
8
30000/30000 [==============================] - 0s 11us/step
9
30000/30000 [==============================] - 0s 11us/step
10
30000/30000 [==============================] - 0s 11us/step
11
30000/30000 [==============================] - 0s 12us/step
12
30000/30000 [==============================] - 0s 11us/step
13
30000/30000 [==============================] - 0s 12us/step
14
30000/30000 [==============================] - 0s 11us/step
15
30000/30000 [==============================] - 0s 11us/step
16
30000/30000 [==============================] - 0s 12us/step
1

30000/30000 [==============================] - 0s 11us/step
132
30000/30000 [==============================] - 0s 11us/step
133
30000/30000 [==============================] - 0s 11us/step
134
30000/30000 [==============================] - 0s 11us/step
135
30000/30000 [==============================] - 0s 11us/step
136
30000/30000 [==============================] - 0s 11us/step
137
30000/30000 [==============================] - 0s 11us/step
138
30000/30000 [==============================] - 0s 11us/step
139
30000/30000 [==============================] - 0s 12us/step
140
30000/30000 [==============================] - 0s 12us/step
141
30000/30000 [==============================] - 0s 11us/step
142
30000/30000 [==============================] - 0s 11us/step
143
30000/30000 [==============================] - 0s 11us/step
144
30000/30000 [==============================] - 0s 11us/step
145
30000/30000 [==============================] - 0s 11us/step
146
30000/30000 [===========================